# Exemplary use cases for the Co2calculator
In this Jupyter notebook you will find some helpful examples of how to use the co2calculator.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
from pathlib import Path
from dotenv import load_dotenv
import os

sys.path.insert(0, Path.cwd().parent.parent.absolute().as_posix())

import co2calculator as co2

from co2calculator.parameters import *

from co2calculator.api.energy import Energy
from co2calculator.api.trip import Trip

In [2]:
load_dotenv()

# Load environment vars
ORS_API_KEY = os.environ.get("ORS_API_KEY")

In [3]:
#print(ORS_API_KEY)

## Examples for the Energy class 

Let's take a look at some examples of the Energy class. It has two methods from_electricity and from_heating. First, you must initialize the Energy object. Depending on the fuel type the consumption can take on different units. <br>
Then we can calculate the emissions for an electricity consumption of 100kWh in Germany.

In [4]:
# Initialize Energy object 
energy=Energy()

# calculate emissions for electricity 
electricity_result=energy.from_electricity(consumption=100, country_code="DE").calculate_co2e()

print(f"The result for electricity calculations is: {electricity_result}\n")

The result for electricity calculations is: EnergyEmissions(co2e=44.912, emission_factor=0.44912, emission_parameters=ElectricityEmissionParameters(category=<EmissionCategory.ELECTRICITY: 'electricity'>, fuel_type=<ElectricityFuel.PRODUCTION_FUEL_MIX: 'production fuel mix'>, country_code='DE', own_share=1.0), consumption=100, unit='kWh')



You can use the same Energy class object to calculate the heating emissions. The default fuel type is gas. Now you only need to set in_kwh to True. Otherwise, you must specify a fuel type.

In [5]:
# calculate emissions for heating 
heating_result=energy.from_heating(consumption=100, in_kwh=True).calculate_co2e()

print(f"The emissions for the heating calculations with 100 kWh of gas are:{heating_result}")

The emissions for the heating calculations with 100 kWh of gas are:EnergyEmissions(co2e=18.0, emission_factor=0.18, emission_parameters=HeatingEmissionParameters(category=<EmissionCategory.HEATING: 'heating'>, fuel_type=<HeatingFuel.GAS: 'gas'>, country_code='global', own_share=1.0), consumption=100, unit='kWh')


If you want to know the respectively unit of each fuel type you can use the method get_options().

In [6]:
# get options for heating
heating_options=energy.from_heating(consumption=100, fuel_type='oil').get_options()

print(f"The options for heating are:{heating_options}")

# calculate emissions of oil
heating_oil=energy.from_heating(consumption=100, fuel_type='oil').calculate_co2e()

print(f"The emissions for the heating calculations with 100 liter of oil are:{heating_oil}")

The options for heating are:      fuel_type unit
0           oil    l
1    liquid gas   kg
2          coal   kg
3  wood pellets   kg
4    wood chips   kg
5           gas  m^3
The emissions for the heating calculations with 100 liter of oil are:EnergyEmissions(co2e=286.20000000000005, emission_factor=0.27, emission_parameters=HeatingEmissionParameters(category=<EmissionCategory.HEATING: 'heating'>, fuel_type=<HeatingFuel.OIL: 'oil'>, country_code='global', own_share=1.0), consumption=100, unit='l')


The user can also provide further information, e.g. the fuel type or how large their share of the energy consumed is. Let's assume you want to calculate the emissions for 100 kg of coal and you share your apartment with 4 other people.

In [7]:
# heating calculations with more user defined input
heating_energy=Energy()

heating_result3=heating_energy.from_heating(consumption=100, fuel_type="coal", own_share=0.25).calculate_co2e()

print(heating_result3)

EnergyEmissions(co2e=52.5, emission_factor=0.35, emission_parameters=HeatingEmissionParameters(category=<EmissionCategory.HEATING: 'heating'>, fuel_type=<HeatingFuel.COAL: 'coal'>, country_code='global', own_share=0.25), consumption=100, unit='kg')


The same applies to electricity. Let's assume you use the normal production fuel mix and share your office with another person.

In [8]:
# electricity calculations with user defined input
electricity_energy=Energy()

electricity_result2=electricity_energy.from_electricity(consumption=100, fuel_type="production fuel mix", own_share=0.5, country_code="DE").calculate_co2e()

print(electricity_result2)

EnergyEmissions(co2e=22.456, emission_factor=0.44912, emission_parameters=ElectricityEmissionParameters(category=<EmissionCategory.ELECTRICITY: 'electricity'>, fuel_type=<ElectricityFuel.PRODUCTION_FUEL_MIX: 'production fuel mix'>, country_code='DE', own_share=0.5), consumption=100, unit='kWh')


## Examples for the Trip class 
You have two options for calculating the emissions for the Trip class. Either you specify the distance in kilometers when initializing the Trip class object, or you define the start and destination of your trip.
### Trip calculations with a given distance
Let's assume you want to make a long journey of 5000 km and want to know which type of transportation causes the lowest emissions.

In [9]:
# Initiate Trip object
trip=Trip(distance=5000)

In [10]:
# trip by train
train_trip=trip.by_train().calculate_co2e()
print(train_trip)

TransportEmissions(co2e=34.0, emission_factor=0.0068, emission_parameters=TrainEmissionParameters(category=<EmissionCategory.TRANSPORT: 'transport'>, subcategory=<TransportationMode.TRAIN: 'train'>, vehicle_range=<BusTrainRange.LONG_DISTANCE: 'long-distance'>, country_code='global'), distance=5000, start=None, start_coords=None, destination=None, destination_coords=None)


For your plane trip you also want to compare the emissions when flying first class versus economy class.

In [11]:
# first_class trip by plane
plane_first_class=trip.by_plane(seating="first_class").calculate_co2e()

print(plane_first_class)

TransportEmissions(co2e=3015.5, emission_factor=0.6031, emission_parameters=PlaneEmissionParameters(category=<EmissionCategory.TRANSPORT: 'transport'>, subcategory=<TransportationMode.PLANE: 'plane'>, seating=<FlightClass.FIRST: 'first_class'>, vehicle_range=<FlightRange.LONG_HAUL: 'long-haul'>), distance=5000, start=None, start_coords=None, destination=None, destination_coords=None)


In [12]:
# economy_class trip by plane
plane_economy_class=trip.by_plane(seating="economy_class").calculate_co2e()

print(plane_economy_class)

TransportEmissions(co2e=947.5, emission_factor=0.1895, emission_parameters=PlaneEmissionParameters(category=<EmissionCategory.TRANSPORT: 'transport'>, subcategory=<TransportationMode.PLANE: 'plane'>, seating=<FlightClass.ECONOMY: 'economy_class'>, vehicle_range=<FlightRange.LONG_HAUL: 'long-haul'>), distance=5000, start=None, start_coords=None, destination=None, destination_coords=None)


Your last option is to go on your big trip in an electric car, and you travel with two of your colleagues in a large car.

In [13]:
# trip with electric car
car=trip.by_car(fuel_type="electric", size="large", passengers=3).calculate_co2e()
print(car)

TransportEmissions(co2e=151.66666666666666, emission_factor=0.091, emission_parameters=CarEmissionParameters(category=<EmissionCategory.TRANSPORT: 'transport'>, subcategory=<TransportationMode.CAR: 'car'>, fuel_type=<CarFuel.ELECTRIC: 'electric'>, size=<Size.LARGE: 'large'>, passengers=3), distance=5000, start=None, start_coords=None, destination=None, destination_coords=None)


### Provide your own emission factor
If you know the emission factor for example of your car, you can use the method by_custom() to calculate the emissions of your trip.

In [14]:
custom_emission_factor = trip.by_custom(emission_factor=0.1).calculate_co2e()
print(custom_emission_factor)

TransportEmissions(co2e=500.0, emission_factor=0.1, emission_parameters={'transportation_mode': None}, distance=5000, start=None, start_coords=None, destination=None, destination_coords=None)


### Trip calculations with input start and destination
Enter the start and destination either as a dictionary or as a string.

In [15]:
# provide start and destination as dictionary
start = {"locality": "Heidelberg", "country": "Germany", "address_type": "address",}
destination = {"locality": "Berlin", "country": "Germany", "address_type": "address",}

# initiate the Trip object with given start and destination
trip = Trip(start=start, destination=destination)

Calculate the emissions of your car journey with calculate_co2e(), this also returns the locations used and all parameters used.

In [16]:
# calculate emissions of trip by car
car_co2e = trip.by_car().calculate_co2e()
print(car_co2e)

4 location(s) found. Using this result: Heidelberg, Germany (data type: locality)
Coords:  [8.769869, 49.37625]
7 location(s) found. Using this result: Berlin, Germany (data type: locality)
Coords:  [13.407032, 52.524932]
TransportEmissions(co2e=119.48486048000001, emission_factor=0.1864, emission_parameters=CarEmissionParameters(category=<EmissionCategory.TRANSPORT: 'transport'>, subcategory=<TransportationMode.CAR: 'car'>, fuel_type=<CarFuel.AVERAGE: 'average'>, size=<Size.AVERAGE: 'average'>, passengers=1), distance=641.0132, start={'locality': 'Heidelberg', 'country': 'Germany'}, start_coords=[8.769869, 49.37625], destination={'locality': 'Berlin', 'country': 'Germany'}, destination_coords=[13.407032, 52.524932])


You can either use the same trip object to calculate emissions for other modes of transport. In this case, the same coordinates are used. Or you can initialize a new trip object with the appropriate address_type, in which case the actual address of the station, for example, is used.

In [17]:
# train trip with the same Trip object
train_co2e = trip.by_train().calculate_co2e()
print(train_co2e)

4 location(s) found. Using this result: Heidelberg, Germany (data type: locality)
Coords:  [8.769869, 49.37625]
7 location(s) found. Using this result: Berlin, Germany (data type: locality)
Coords:  [13.407032, 52.524932]
TransportEmissions(co2e=3.8956776670447653, emission_factor=0.0068, emission_parameters=TrainEmissionParameters(category=<EmissionCategory.TRANSPORT: 'transport'>, subcategory=<TransportationMode.TRAIN: 'train'>, vehicle_range=<BusTrainRange.LONG_DISTANCE: 'long-distance'>, country_code='global'), distance=572.8937745654067, start={'locality': 'Heidelberg', 'country': 'Germany'}, start_coords=[8.769869, 49.37625], destination={'locality': 'Berlin', 'country': 'Germany'}, destination_coords=[13.407032, 52.524932])


In [18]:
# plane trip with the same Trip object
plane_co2e = trip.by_plane().calculate_co2e()
print(plane_co2e)

4 location(s) found. Using this result: Heidelberg, Germany (data type: locality)
Coords:  [8.769869, 49.37625]
7 location(s) found. Using this result: Berlin, Germany (data type: locality)
Coords:  [13.407032, 52.524932]
TransportEmissions(co2e=182.71374403439816, emission_factor=0.3192, emission_parameters=PlaneEmissionParameters(category=<EmissionCategory.TRANSPORT: 'transport'>, subcategory=<TransportationMode.PLANE: 'plane'>, seating=<FlightClass.AVERAGE: 'average'>, vehicle_range=<FlightRange.SHORT_HAUL: 'short-haul'>), distance=572.4114788045056, start={'locality': 'Heidelberg', 'country': 'Germany'}, start_coords=[8.769869, 49.37625], destination={'locality': 'Berlin', 'country': 'Germany'}, destination_coords=[13.407032, 52.524932])


If you want to use the exact location of the trainstation or airport, your start and destination input could look like this.

In [19]:
# give start and destination as dictionary for train
start = {"station_name": "Heidelberg Hbf", "country": "DE", "address_type": "trainstation",}
destination = {"station_name": "Berlin Hbf", "country": "DE", "address_type": "trainstation",}

# initiate train Trip object with given start and destination
train_trip = Trip(start=start, destination=destination)

# calculate emissions of trip by train
train_co2e = train_trip.by_train().calculate_co2e()
print(train_co2e)

TransportEmissions(co2e=3.9030595999198807, emission_factor=0.0068, emission_parameters=TrainEmissionParameters(category=<EmissionCategory.TRANSPORT: 'transport'>, subcategory=<TransportationMode.TRAIN: 'train'>, vehicle_range=<BusTrainRange.LONG_DISTANCE: 'long-distance'>, country_code='global'), distance=573.9793529293943, start={'station_name': 'Heidelberg Hbf', 'country': 'DE'}, start_coords=(8.675741, 49.403779), destination={'station_name': 'Berlin Hbf', 'country': 'DE'}, destination_coords=(13.386987, 52.520501))


Travelling by plane your code could look like this. 

In [20]:
# start and destination for a trip by plane
start = {"IATA": "FRA", "address_type": "airport",}
destination = {"IATA": "BER", "address_type": "airport",}

# initiate plane Trip object with given start and destination
plane_trip = Trip(start=start, destination=destination)

# calculate emissions for trip by plane
plane_co2e = plane_trip.by_plane().calculate_co2e()
print(plane_co2e)

TransportEmissions(co2e=167.82950702986088, emission_factor=0.3192, emission_parameters=PlaneEmissionParameters(category=<EmissionCategory.TRANSPORT: 'transport'>, subcategory=<TransportationMode.PLANE: 'plane'>, seating=<FlightClass.AVERAGE: 'average'>, vehicle_range=<FlightRange.SHORT_HAUL: 'short-haul'>), distance=525.7816636273838, start={'IATA': 'FRA'}, start_coords=[8.561096, 50.030241], destination={'IATA': 'BER'}, destination_coords=[13.500672, 52.362247])
